# Door codes

After a fairly dull flight, you've finally arrived at your hotel. The good news is that the hotel has high-security electronic locks on the room doors. The bad news is that the staff are rather busy, and you think it will take a long time to get to your room.

Luckily, you know how their system works. 

Each door in the hotel has a keyboard on the lock. You have to enter the correct two-letter code to get in to the room. Because the staff know that people won't remember the codes, they tell you a pass phrase you can use the generate the code from.

There's a long queue for people to be told how to generate the code from the pass phrase. You were here last year and you still remember how to do it.

You start with the first two letters of the pass phrase. This is the starting value of your code.

Then, for each subsequent letter in the pass phrase, you:
1. "Add" the second letter of the code to the first letter of the code, replacing the first letter of the code
2. "Add" the current letter of the pass phrase to the second letter of the code, replacing the second letter of the code
3. Move on to the next letter of the pass phrase

"Adding" letters is done by converting the letters to their position in the alphabet (starting at one), adding, then converting the number back to a letter. Numbers greater than 26 are "wrapped around". For instance, to add `u` + `k`, convert them to numbers (`u`=`21`, `k`=`11`), add them (`21` + `11` = `32`), then convert back to a letter (`32` is larger than 26, so it becomes `32` - 26 = `6`, which is `f`).

Anything that isn't a lower-case letter is ignored.

For example, to find the code from the pass phrase `the cat`, the code starts as being the first two letters `th`, then the subsequent letters are used to update the code as follows:

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| th | (20, 8) | e | 5 | 28 | 13 | bm |
| bm | (2, 13) | c | 3 | 15 | 16 | op |
| op | (15, 16) | a | 1 | 31 | 17 | eq |
| eq | (5, 17) | t | 20 | 22 | 37 | vk |

So the final code is `vk`

## Part 1
Your passphrase is 
"the traveller in the grey riding-coat, who called himself mr. melville, was contemplating the malice of which the gods are capable."

**What is your door code?**

In [98]:
import string

In [99]:
def o(letter):
    return ord(letter) - ord('a') + 1

def c(number):
    return chr((number - 1) % 26 + ord('a'))

In [100]:
def sanitise(phrase):
    return ''.join(l for l in phrase if l in string.ascii_lowercase)

In [101]:
def whash1(word, show_steps=False):
    if show_steps:
        print('| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |'
              'new second<br/>part of code | new code |')
        print('|:---|:---|:---|:---|:---|:---|:---|')
    h = list(word[:2])
    for l in word[2:]:
        if show_steps:
            h_before = h.copy()
        h[0] = c(o(h[0]) + o(h[1]))
        h[1] = c(o(h[1]) + o(l))
        if show_steps:
            print('| {} | {} | {} | {} | {} | {} | {} |'.format(
                  ''.join(h_before), 
                  (o(h_before[0]), o(h_before[1])),
                  l, 
                  o(l), 
                  o(h_before[0]) + o(h_before[1]), 
                  o(h_before[1]) + o(l), 
                  ''.join(h)))
    return ''.join(h)

In [102]:
whash1(sanitise('the cat'), show_steps=True)

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| th | (20, 8) | e | 5 | 28 | 13 | bm |
| bm | (2, 13) | c | 3 | 15 | 16 | op |
| op | (15, 16) | a | 1 | 31 | 17 | eq |
| eq | (5, 17) | t | 20 | 22 | 37 | vk |


'vk'

In [103]:
passphrase = sanitise("the traveller in the grey riding-coat, who called himself mr. melville, was "
                      "contemplating the malice of which the gods are capable.")

In [104]:
whash1(passphrase)

'mc'

You arrive at your door but the passcode doesn't work!

Looking at the lock, you see it's a different model from last time you were here. Reluctantly, you return to the hotel front desk and join the queue for instructions. 

Indeed, the algorithm has been changed, "to improve security," they say.

The new algorithm starts with the same initial value of the code for every passphrase. In in this case, the initial code is `ri`. The algorithm also uses two contants, `m1`= 5 and `m2` = 11.

Then, for each letter in the pass phrase, the code is updated as follows:

1. "Add" the second letter of the code to the first letter of the code "multiplied by" `m1`, replacing the first letter of the code
2. "Add" the current letter of the pass phrase to the second letter of the code "multiplied by" `m2`, replacing the second letter of the code

"Multiplying" letters is done by converting the letters to their position in the alphabet (starting at one) and multiplying. For instance, to multiply `u` by 11, convert `u` to `21`, multiply by 11 (`21` × `11` = `231`), then convert back to a letter (`231` is larger than 26, so it becomes `22`, which is `w`).

Again, anything that isn't a lower-case letter is ignored.

For example, to find the code from the pass phrase `the cat`, the code starts as being the first two letters `th`, then the subsequent letters are used to update the code as follows:

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| ri | (18, 9) | t | 20 | 63 | 229 | ku |
| ku | (11, 21) | h | 8 | 116 | 109 | le |
| le | (12, 5) | e | 5 | 37 | 60 | kh |
| kh | (11, 8) | c | 3 | 51 | 41 | yo |
| yo | (25, 15) | a | 1 | 100 | 26 | vz |
| vz | (22, 26) | t | 20 | 152 | 246 | vl |

So the final code is `pl`

## Part 2
Your passphrase remains "the traveller in the grey riding-coat, who called himself mr. melville, was contemplating the malice of which the gods are capable."

Using this new algorithm, **what is your door code?**

In [105]:
o('u'), o('u') * 11, (o('u') * 11 - 1) % 26, c(o('u') * 11)

(21, 231, 22, 'w')

In [113]:
def whash2(word, h0=None, m1=5, m2=11, show_steps=False):
    if show_steps:
        print('| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |'
              'new second<br/>part of code | new code |')
        print('|:---|:---|:---|:---|:---|:---|:---|')
    if h0 is None:
        h = list('ri')
    else:
        h = list(h0)
    for l in word:
        if show_steps:
            h_before = h.copy()
        h[0] = c(o(h[0]) + o(h[1]) * m1)
        h[1] = c(o(h[1]) + o(l) * m2)
        if show_steps:
            print('| {} | {} | {} | {} | {} | {} | {} |'.format(
                  ''.join(h_before), 
                  (o(h_before[0]), o(h_before[1])),
                  l, 
                  o(l), 
                  o(h_before[0]) + o(h_before[1]) * m1, 
                  o(h_before[1]) + o(l) * m2, 
                  ''.join(h)))
    return ''.join(h)

In [116]:
whash2(sanitise('the cat'), show_steps=True)

| old code | code as<br>numbers | passphrase<br/>letter | number of<br/>letter | new first<br/>part of code |new second<br/>part of code | new code |
|:---|:---|:---|:---|:---|:---|:---|
| ri | (18, 9) | t | 20 | 63 | 229 | ku |
| ku | (11, 21) | h | 8 | 116 | 109 | le |
| le | (12, 5) | e | 5 | 37 | 60 | kh |
| kh | (11, 8) | c | 3 | 51 | 41 | yo |
| yo | (25, 15) | a | 1 | 100 | 26 | vz |
| vz | (22, 26) | t | 20 | 152 | 246 | vl |


'vl'

In [117]:
whash2(passphrase)

'qb'